In [ ]:
import seaborn as sns 
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


df = sns.load_dataset("iris")

In [ ]:
X = df.drop('species', axis=1)
y = df['species'] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, random_state=777) 

## Задание 1
Среди заданного набора параметров для метода ближайших соседей найдите такие параметры, при которых качество алгоритма (accuracy) оказывается наилучшим

In [ ]:
def square_distance(x):
    return 1/x*2

params_grid = {
    'n_neighbors': range(1,100,2), # Перебираем от 1 до 100 с шагом 2
    'weights': ['uniform', 'distance', square_distance], # с каким весом берём каждого соседа. uniform - с одинаковым, distance - обратно пропорционально расстоянию. Ещё можно передать свою функцию
    'algorithm': ['auto'], # алгоритм поиска ближайших соседей. Влияет только на скорость обучения
    'leaf_size': [30], # параметры некоторых алгоритмов поиска ближ соседей. Влияет только на скорость обучения
    'metric': ['minkowski'], # метрика расстояния, используемая в алгоритме поиска ближайших соседей
    'p': [2] # параметр метрики, используемой в методе ближайших соседей 
}        

In [ ]:
best_params = {}
best_acc = 0

for n_neighbours in params_grid['n_neighbors']:
    for weights in params_grid['weights']:
        for alg in params_grid['algorithm']:
            for leaf_size in params_grid['leaf_size']:
                for metric in params_grid['metric']:
                    for p in params_grid['p']:
                        params = {'n_neighbors': n_neighbours,
                                  'weights': weights,
                                  'algorithm': alg,
                                  'leaf_size': leaf_size,
                                  'metric': metric,
                                  'p': p }
                        clf = KNeighborsClassifier(**params)
                        clf.fit(X_train, y_train)
                        y_pred = clf.predict(X_test)
                        acc = accuracy_score(y_test, y_pred)
                        if acc > best_acc:
                            best_acc = acc
                            best_params = params

print('Best accuracy: %f' % best_acc)
print(best_params)

## Задание 2

Предсказание выживших титаника. Загрузите файл и по выбранным вами числовым признакам постройте наилучшую модель ближайших соседей для предсказания колонки *Survived*

In [ ]:
import pandas as pd 
data = pd.read_csv('../data/Titanic.csv')
data.head()

In [ ]:
data.shape

In [ ]:
print('All objects', data.shape[0])
for col in data.columns:
    print('{:11} {:3}'.format(col, data[col].nunique()))

In [ ]:
data.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

In [ ]:
data.info()

In [ ]:
cat_cols = [col for col in data.columns if data[col].dtype == 'object']

In [ ]:
data[cat_cols] = data[cat_cols].fillna('NoneType')

In [ ]:
for col in cat_cols:
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.transform(data[col])

In [ ]:
X, y = data.drop('Survived', axis=1), data['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    test_size=0.2, random_state=777) 

In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
X_train = X_train.fillna(X_train.mean()).astype(float)
X_test = X_test.fillna(X_test.mean()).astype(float)

In [ ]:
ms = MinMaxScaler()
ms.fit(X_train)
X_train = pd.DataFrame(ms.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(ms.transform(X_test), columns=X_test.columns)
#Здесь не обязательно пересоздавать датафрейм, но пока данные маленькие - так нагляднее.

In [ ]:
X_train.head()

In [ ]:
# features_list = в трейн сете остались только нужные колонки

Добейтесь наилучшего качества (accuracy) на тестовой выборке

In [ ]:
params_grid = {
    'n_neighbors': range(1,100,2), # Перебираем от 1 до 100 с шагом 2
    'weights': ['uniform', 'distance'], # с каким весом берём каждого соседа. uniform - с одинаковым, distance - обратно пропорционально расстоянию. Ещё можно передать свою функцию
    'algorithm': ['auto'], # алгоритм поиска ближайших соседей. Влияет только на скорость обучения
    'leaf_size': [30], # параметры некоторых алгоритмов поиска ближ соседей. Влияет только на скорость обучения
    'metric': ['minkowski'], # метрика расстояния, используемая в алгоритме поиска ближайших соседей
    'p': [2] # параметр метрики, используемой в методе ближайших соседей 
}   

In [ ]:
best_params = {}
best_acc = 0

for n_neighbours in params_grid['n_neighbors']:
    for weights in params_grid['weights']:
        for alg in params_grid['algorithm']:
            for leaf_size in params_grid['leaf_size']:
                for metric in params_grid['metric']:
                    for p in params_grid['p']:
                        params = {'n_neighbors': n_neighbours,
                                  'weights': weights,
                                  'algorithm': alg,
                                  'leaf_size': leaf_size,
                                  'metric': metric,
                                  'p': p }
                        clf = KNeighborsClassifier(**params)
                        clf.fit(X_train, y_train)
                        y_pred = clf.predict(X_test)
                        acc = accuracy_score(y_test, y_pred)
                        if acc > best_acc:
                            best_acc = acc
                            best_params = params

print('Best accuracy: %f' % best_acc)
print(best_params)